<a href="https://colab.research.google.com/github/AndreVelasques7/inteligencia_computacional/blob/main/ed-03/ed_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from dataclasses import dataclass
from typing import Callable
from random import randint, sample, random
from pprint import pprint

@dataclass
class Cromossomo:
    """Representa um cromossomo no algoritmo genético."""
    dados: list[int]  # Representa os genes do cromossomo
    fitness: float = 0  # Aptidão deste cromossomo

@dataclass
class Config:
    """Configurações para a execução do algoritmo genético."""
    tam_cromossomo: int  # Tamanho dos cromossomos
    tam_populacao: int  # Tamanho da população
    fitness: Callable[[Cromossomo], float]  # Função de avaliação

    # Função para selecionar os pais para cruzamento
    selecionar_pais: Callable[[list[Cromossomo]], list[tuple[Cromossomo, Cromossomo]]]

    # Função para aplicar cruzamento entre pais
    aplicar_cruzamento: Callable[[list[tuple[Cromossomo, Cromossomo]]], list[Cromossomo]]

    # Função para aplicar mutação nos cromossomos
    aplicar_mutacao: Callable[[list[Cromossomo], float], None]
    taxa_mutacao: float  # Taxa de mutação

    # Função para selecionar os sobreviventes da próxima geração
    selecionar_sobreviventes: Callable[[list[Cromossomo]], list[Cromossomo]]

def inicializar_populacao(tam_populacao: int, tam_cromossomo: int) -> list[Cromossomo]:
    """Inicializa uma população de cromossomos com genes aleatórios."""
    return [Cromossomo([randint(0, 1) for _ in range(tam_cromossomo)]) for _ in range(tam_populacao)]

def torneio(populacao: list[Cromossomo], tam_torneio: int = 3) -> list[tuple[Cromossomo, Cromossomo]]:
    """Seleciona pais para cruzamento usando o método do torneio."""
    casais = []
    for _ in range(len(populacao) // 2):
        pai1 = max(sample(populacao, tam_torneio * 2), key=lambda x: x.fitness)
        populacao.remove(pai1)

        pai2 = max(sample(populacao, tam_torneio), key=lambda x: x.fitness)
        casais.append((pai1, pai2))
    return casais

def crossover_1_corte(casais: list[tuple[Cromossomo, Cromossomo]]) -> list[Cromossomo]:
    """Aplica o cruzamento de um ponto entre os pais selecionados."""
    filhos = []
    for pai1, pai2 in casais:
        corte = randint(1, len(pai1.dados) - 1)
        filhos.extend([
            Cromossomo(pai1.dados[:corte] + pai2.dados[corte:]),
            Cromossomo(pai2.dados[:corte] + pai1.dados[corte:])
        ])
    return filhos

def mutacao(cromossomos: list[Cromossomo], taxa_mutacao: float):
    """Aplica mutação em um cromossomo com base em uma taxa de mutação."""
    for cromossomo in cromossomos:
        for i, gene in enumerate(cromossomo.dados):
            if random() < taxa_mutacao:
                cromossomo.dados[i] = 1 - gene  # Inverte o gene (0 vira 1 e 1 vira 0)

def elitismo(populacao: list[Cromossomo]) -> list[Cromossomo]:
    """Seleciona a metade mais apta da população."""
    return sorted(populacao, key=lambda x: x.fitness, reverse=True)[:len(populacao)//2]

def maximizar_1s(cromossomo: Cromossomo) -> int:
    """Função de avaliação que busca maximizar o número de 1s no cromossomo."""
    return sum(cromossomo.dados)

def algoritmo_genetico(config: Config) -> list[int]:
    """Executa o algoritmo genético com base nas configurações fornecidas."""
    t = 0
    P = inicializar_populacao(config.tam_populacao, config.tam_cromossomo)

    while True:  # Critério de parada pode ser adicionado aqui
        print("\n> Avaliação da população")
        for c in P:
            c.fitness = config.fitness(c)
            print(c)

        print("\n> Seleção dos pais")
        casais = config.selecionar_pais(P)
        for c in casais:
            print(c)

        print("\n> Crossover e mutação")
        F = config.aplicar_cruzamento(casais)
        config.aplicar_mutacao(F, config.taxa_mutacao)
        for c in F:
            print(c)

        print("\n> Avaliação da nova população")
        for c in F:
            c.fitness = config.fitness(c)
            print(c)

        print("\n> Selecionar sobreviventes")
        P = config.selecionar_sobreviventes(P + F)
        for c in P:
            print(c)

        t += 1
        break  # Este é um placeholder; em uma implementação real, você deve definir um critério de parada adequado.


config = Config(
    tam_cromossomo=8,
    tam_populacao=10,
    fitness=maximizar_1s,
    taxa_mutacao=0.05,
    selecionar_pais=torneio,
    aplicar_cruzamento=crossover_1_corte,
    aplicar_mutacao=mutacao,
    selecionar_sobreviventes=elitismo,
)
solucao = algoritmo_genetico(config)



> Avaliação da população
Cromossomo(dados=[0, 0, 0, 1, 0, 0, 0, 0], fitness=1)
Cromossomo(dados=[0, 1, 1, 0, 1, 0, 0, 0], fitness=3)
Cromossomo(dados=[1, 1, 0, 1, 0, 0, 0, 0], fitness=3)
Cromossomo(dados=[1, 1, 1, 1, 1, 0, 1, 1], fitness=7)
Cromossomo(dados=[0, 0, 0, 0, 0, 0, 1, 0], fitness=1)
Cromossomo(dados=[1, 1, 1, 1, 1, 0, 0, 1], fitness=6)
Cromossomo(dados=[0, 1, 1, 0, 0, 1, 1, 0], fitness=4)
Cromossomo(dados=[1, 0, 1, 0, 0, 0, 0, 0], fitness=2)
Cromossomo(dados=[1, 0, 0, 1, 1, 1, 0, 1], fitness=5)
Cromossomo(dados=[0, 0, 0, 1, 1, 0, 1, 1], fitness=4)

> Seleção dos pais
(Cromossomo(dados=[1, 1, 1, 1, 1, 0, 1, 1], fitness=7), Cromossomo(dados=[1, 1, 1, 1, 1, 0, 0, 1], fitness=6))
(Cromossomo(dados=[1, 1, 1, 1, 1, 0, 0, 1], fitness=6), Cromossomo(dados=[0, 1, 1, 0, 0, 1, 1, 0], fitness=4))
(Cromossomo(dados=[0, 1, 1, 0, 0, 1, 1, 0], fitness=4), Cromossomo(dados=[0, 1, 1, 0, 1, 0, 0, 0], fitness=3))
(Cromossomo(dados=[0, 0, 0, 1, 1, 0, 1, 1], fitness=4), Cromossomo(dados=[1, 0, 0